# Scraping protocol descriptions

### Import packages

# Setup

In [1]:
from pathlib import Path
import pickle
import pandas as pd
import requests
import numpy as np


In [22]:
from tabula import read_pdf


In [ ]:
# Data downloaded from the following link:
link = 'https://www.in.gov/omb/2664.htm'

# Define functions

In [104]:
def get_pdf(row):
    url = row.url
    filename = row.school_name + '.pdf'
    cmd  = f"""curl {url} > 'pdfs/{filename}'"""
    !{cmd}
#     print(cmd)

def get_schools(filename,df_in,expenditure_type,unique_cols):

    useful_cols = ['year',*unique_cols]
    df_in = df_in.rename(columns = df_in.iloc[0,:])
    df_in['year']  = df_in.loc[:,df_in.apply(lambda col: bool(col.str.contains('FY.*20[0-9]{,2}').sum()))]
    df_in = (df_in
     .loc[df_in.year.notnull(),useful_cols]
     .assign(year = lambda df: df.year.str.extract('.*(20[0-9]{2,2}).*',expand=False))
     .apply(lambda col:col.str.split(" ").str[0] )
     .apply(lambda col:col.str.replace('\D',''))
     .assign(expenditure_type = expenditure_type,
            school = Path(filename).stem)

    )
    df_in = pd.melt(
        df_in,
        id_vars=[*df_in.columns.drop(useful_cols[1:])],
        value_name='amount',
        var_name = 'funding_category'
        )
    return df_in

def get_spending_breakdown(filename):

    dfs_all = read_pdf(filename,multiple_tables=True,pages=1)


    unique_cols = ['Student Academic Achievement','Student Instructional Support']
    expenditure_type = "Instructional"
    df_1 = get_schools(filename,dfs_all[0].T,expenditure_type,unique_cols)

    unique_cols = ['Overhead and Operational','Non Operational','Not Categorized']
    expenditure_type = "Non-Instructional"
    df_2 = get_schools(filename,dfs_all[1].T,expenditure_type,unique_cols)
    df_full = pd.concat([df_1,df_2], axis = 0)

    df_full
    return df_full

# Get pdfs

In [163]:
!mkdir pdfs

In [ ]:
df_pdfs = pd.read_csv('school_pdfs.csv')

df_pdfs['school_name'] = (df_pdfs
                          .school_name.str.replace('(?<!;) +?','_')
                          .str.replace(';','_')
                          .str.replace('[^a-zA-Z0-9_]','')
                         
                         )

# get_pdf(df_pdfs.iloc[1,:])
df_pdfs.apply(lambda row:get_pdf(row), axis=1)

df_pdfs['filename'] = df_pdfs.school_name.apply(lambda x: 'pdfs/' + x + '.pdf')

In [ ]:
df_pdfs.head()

# Process pdfs

In [114]:
dfs = []
bad_files = []
for filename in df_pdfs.filename:
    try:
        df_spending = get_spending_breakdown(filename)
        dfs.append(df_spending)
    except:
        bad_files.append(filename)

Error: 
Error: 
Error: 
Error: 
Error: 
Error: 


In [115]:
len(dfs)

374

In [117]:
bad_files

['pdfs/Career_Academy_Middle_School_9965httpswwwingovombfiles9965fy16.pdf',
 'pdfs/Christel_House_DORSnbsp_9385.pdf',
 'pdfs/ombfiles3710fy16_target_blankCrothersville_Community_Schoolsnbsp_3710.pdf',
 'pdfs/Daleville_Community_Schoolsnbsp_1940.pdf',
 'pdfs/Fayette_County_School_Corpnbsp_2395.pdf',
 'pdfs/ombfilesmonrowgregg5900_target_blankMonroeGregg_School_Districtnbsp_5900.pdf']

In [118]:
df_all_schools = pd.concat(dfs,axis = 0)

In [119]:
len(df_all_schools)

11220

In [120]:
df_all_schools.to_csv('all_school_funding.csv',index=False)

# Do further analysis

In [2]:
df_all_schools = pd.read_csv('all_school_funding.csv')

In [5]:
df_all_schools['percentage'] = df_all_schools.groupby(['school','year'])['amount'].transform(lambda x: 100 * (x / x.sum()))

In [7]:
df_all_schools.sort_values(['school','year']).head(100)

,year,expenditure_type,school,funding_category,amount,percentage
0,2006,Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Student Academic Achievement,885297,35.119113
6,2006,Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Student Instructional Support,191398,7.592625
12,2006,Non-Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Overhead and Operational,649896,25.780920
18,2006,Non-Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Non Operational,794250,31.507342
24,2006,Non-Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Not Categorized,0,0.000000
1,2009,Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Student Academic Achievement,1112059,74.739267
7,2009,Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Student Instructional Support,375286,25.222223
13,2009,Non-Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Overhead and Operational,258,0.017340
19,2009,Non-Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Non Operational,315,0.021171
25,2009,Non-Instructional,21st_Century_Charter_Sch_of_Garynbsp_9545,Not Categorized,0,0.000000
